# 07 - Neural Network LPPLS Methods

Two neural network approaches to LPPLS bubble detection from Nielsen, Sornette & Raissi (2024):

1. **M-LNN** (Mono-LPPLS NN) — one network trained per series, minimizes reconstruction MSE
2. **P-LNN** (Poly-LPPLS NN) — pre-trained on 100K synthetic series, ~700x faster than CMA-ES

Both share a physics-informed design: the network predicts nonlinear LPPLS
parameters (tc, m, omega), then linear parameters (A, B, C1, C2) are solved
analytically via OLS. This makes training stable and predictions interpretable.

**Reference:** Nielsen, Sornette & Raissi (2024), [arXiv:2405.12803](https://arxiv.org/abs/2405.12803)

**Requirements:** `pip install fatcrash[deep]` (adds PyTorch)

> **DISCLAIMER:** This software is for academic research and educational purposes only.
> It does not constitute financial advice.

## Imports

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from fatcrash.data.ingest import from_sample
from fatcrash.data.transforms import log_prices, time_index
from fatcrash.indicators.lppls_indicator import fit_lppls

plt.style.use("seaborn-v0_8-whitegrid")
plt.rcParams["figure.figsize"] = (14, 6)

## 1. Load BTC data

In [ ]:
df = from_sample("btc")
df = time_index(df)
df["log_price"] = log_prices(df["close"].values)

# Focus on the 2017 bubble for demonstrations
bubble_mask = (df.index >= "2017-01-01") & (df.index <= "2018-02-06")
bubble_df = df[bubble_mask].copy()

times = np.arange(len(bubble_df), dtype=np.float64)
lp = bubble_df["log_price"].values

print(f"Bubble window: {len(bubble_df)} days ({bubble_df.index[0].date()} to {bubble_df.index[-1].date()})")

fig, ax = plt.subplots(figsize=(14, 5))
ax.plot(bubble_df.index, bubble_df["close"], color="steelblue")
ax.set_title("BTC/USD — 2017 Bubble")
ax.set_ylabel("Price (USD)")
plt.tight_layout()
plt.show()

## 2. M-LNN — Mono-LPPLS Neural Network

One small network (2 hidden layers, 64 units) trained **per time series**.
The network predicts (tc, m, omega) and the loss is the MSE between the LPPLS
reconstruction and observed prices. No pre-training needed.

Architecture: `Linear(N, 64) → ReLU → Linear(64, 64) → ReLU → Linear(64, 3)`

In [ ]:
from fatcrash.nn.mlnn import fit_mlnn

mlnn_result = fit_mlnn(times, lp, epochs=200, lr=1e-2, seed=42)

print("=== M-LNN Result ===")
print(f"  tc       = {mlnn_result.tc:.1f} (day index)")
print(f"  m        = {mlnn_result.m:.4f}")
print(f"  omega    = {mlnn_result.omega:.4f}")
print(f"  B        = {mlnn_result.b:.6f}")
print(f"  RSS      = {mlnn_result.rss:.6f}")
print(f"  is_bubble = {mlnn_result.is_bubble}")
print(f"  confidence = {mlnn_result.confidence:.4f}")

## 3. Classical LPPLS for comparison

CMA-ES optimizer in Rust — the traditional approach.

In [ ]:
classical = fit_lppls(times, lp)

print("=== Classical LPPLS (CMA-ES) ===")
print(f"  tc       = {classical.tc:.1f}")
print(f"  m        = {classical.m:.4f}")
print(f"  omega    = {classical.omega:.4f}")
print(f"  B        = {classical.b:.6f}")
print(f"  RSS      = {classical.rss:.6f}")
print(f"  is_bubble = {classical.is_bubble}")

print("\n=== Comparison ===")
print(f"  {'Param':<10s} {'Classical':>12s} {'M-LNN':>12s}")
print(f"  {'-'*34}")
print(f"  {'tc':<10s} {classical.tc:>12.1f} {mlnn_result.tc:>12.1f}")
print(f"  {'m':<10s} {classical.m:>12.4f} {mlnn_result.m:>12.4f}")
print(f"  {'omega':<10s} {classical.omega:>12.4f} {mlnn_result.omega:>12.4f}")
print(f"  {'RSS':<10s} {classical.rss:>12.6f} {mlnn_result.rss:>12.6f}")

## 4. P-LNN — Poly-LPPLS Neural Network

Pre-trained on synthetic LPPLS data, ~700x faster than CMA-ES at inference.
A deeper network (4 hidden layers: 512→256→128→64) maps min-max normalized
prices to (tc, m, omega) in a single forward pass.

We train on a small dataset here for demonstration. For production use,
train on 100K+ samples.

In [ ]:
from fatcrash.nn.plnn import train_plnn, predict_plnn
from fatcrash.nn.synthetic import generate_dataset
import time

# Train P-LNN on a small synthetic dataset (1000 samples for demo speed)
print("Training P-LNN on 1000 synthetic LPPLS series...")
t0 = time.time()
plnn_model = train_plnn(
    variant="P-LNN-demo",
    n_samples=1000,
    n_obs=252,
    batch_size=8,
    epochs=5,
    lr=1e-4,
    seed=42,
)
train_time = time.time() - t0
print(f"Training took {train_time:.1f}s")

# Inference on real data
t0 = time.time()
plnn_result = predict_plnn(plnn_model, times, lp, window_size=252)
infer_time = time.time() - t0

print(f"\n=== P-LNN Result (inference: {infer_time*1000:.1f}ms) ===")
print(f"  tc       = {plnn_result.tc:.1f}")
print(f"  m        = {plnn_result.m:.4f}")
print(f"  omega    = {plnn_result.omega:.4f}")
print(f"  B        = {plnn_result.b:.6f}")
print(f"  RSS      = {plnn_result.rss:.6f}")
print(f"  is_bubble = {plnn_result.is_bubble}")
print(f"  confidence = {plnn_result.confidence:.4f}")

## 5. Synthetic data visualization

The P-LNN is trained on synthetic LPPLS series with controlled noise.
Let's visualize what the training data looks like.

In [ ]:
from fatcrash.nn.synthetic import generate_lppls_series, add_white_noise, add_ar1_noise

rng = np.random.default_rng(42)
fig, axes = plt.subplots(2, 3, figsize=(16, 8))

for i in range(3):
    clean, params = generate_lppls_series(n_obs=252, rng=rng)
    noisy_white = add_white_noise(clean, alpha=0.05, rng=rng)
    noisy_ar1 = add_ar1_noise(clean, phi=0.9, amplitude=0.03, rng=rng)

    axes[0, i].plot(clean, "k-", linewidth=0.8, label="Clean")
    axes[0, i].plot(noisy_white, "b-", alpha=0.5, linewidth=0.5, label="+ White noise")
    axes[0, i].set_title(f"tc={params['tc']:.0f}, m={params['m']:.2f}, ω={params['omega']:.1f}")
    if i == 0:
        axes[0, i].legend(fontsize=8)
        axes[0, i].set_ylabel("White noise")

    axes[1, i].plot(clean, "k-", linewidth=0.8, label="Clean")
    axes[1, i].plot(noisy_ar1, "r-", alpha=0.5, linewidth=0.5, label="+ AR(1) noise")
    if i == 0:
        axes[1, i].legend(fontsize=8)
        axes[1, i].set_ylabel("AR(1) noise")

fig.suptitle("Synthetic LPPLS Training Data", fontsize=13)
plt.tight_layout()
plt.show()

In [ ]:
from fatcrash.aggregator.signals import (
    mlnn_signal, plnn_signal, aggregate_signals,
)

# Convert NN results to [0, 1] signals
s_mlnn = mlnn_signal(mlnn_result.confidence, mlnn_result.is_bubble)
s_plnn = plnn_signal(plnn_result.confidence, plnn_result.is_bubble)

print("=== NN Signal Values ===")
print(f"  M-LNN signal  = {s_mlnn:.4f} (is_bubble={mlnn_result.is_bubble})")
print(f"  P-LNN signal  = {s_plnn:.4f} (is_bubble={plnn_result.is_bubble})")

# Example aggregation with just the NN signals
components = {
    "mlnn_signal": s_mlnn,
    "plnn_signal": s_plnn,
}
crash = aggregate_signals(components)
print(f"\n=== Aggregated (NN only) ===")
print(f"  probability = {crash.probability:.4f}")
print(f"  level       = {crash.level}")
print(f"  n_agreeing  = {crash.n_agreeing} categories")

In [ ]:
import tempfile, os
from fatcrash.nn.plnn import load_plnn

with tempfile.TemporaryDirectory() as tmpdir:
    # P-LNN save/load
    plnn_path = os.path.join(tmpdir, "plnn_demo.pt")
    plnn_model.save(plnn_path)
    plnn_loaded = load_plnn(plnn_path)
    r = predict_plnn(plnn_loaded, times, lp, window_size=252)
    print(f"P-LNN save/load roundtrip OK — tc={r.tc:.1f}, m={r.m:.4f}")